In [1]:
import subprocess
import pandas as pd
import os

In [2]:
from eval import calculate_nad, calculate_metrics
import joblib
import json
from adat.models.classification_model import LogisticRegressionOnTfIdf

In [3]:
from sklearn.model_selection import ParameterSampler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import numpy as np
import itertools
import time

In [96]:

from adat.utils import load_weights
from adat.utils import calculate_perplexity
from adat.dataset import (
    ClassificationReader,
    CopyNetReader,
    LevenshteinReader,
    LanguageModelingReader,
    END_SYMBOL,
    START_SYMBOL
)
from adat.models import Task, get_model_by_name
from allennlp.data.vocabulary import Vocabulary

from adat.models import get_model_by_name
from adat.utils import load_weights, get_args_from_path
import os
from allennlp.data.vocabulary import Vocabulary
from adat.dataset import ClassificationReader, CopyNetReader, IDENTITY_TOKEN
from adat.predictors.classifier import ClassifierPredictor

In [142]:
for dataset in ['kaggle_transactions', 'ag_news', 'insurance', 'ai_academy_data']:
#for dataset in ['ag_news']:
#for dataset in ['ai_academy_data']:
#for dataset in ['insurance']:
#for dataset in ['kaggle_transactions']:
    base_dir = f'results/{dataset}'
    model_with_weights = joblib.load(f'results/{dataset}/logit_tfidf.model')
    model = model_with_weights['model']
    w = model_with_weights['weights']
    reader = LanguageModelingReader()
    language_model_path = f'experiments/lm_models/{dataset}'
    lm_path = os.path.join(language_model_path, 'best.th')
    lm_vocab_path = os.path.join(language_model_path, 'vocab')
    vocab = Vocabulary.from_files(lm_vocab_path)
    lm_model = get_model_by_name('lm', vocab=vocab)
    load_weights(lm_model, lm_path)

    for method in ['mcmc', 'random', 'cascada', 'hotflip']:
    #for method in ['hotflip']:
        base = os.path.join(base_dir, method)
        for folder in os.listdir(base):
            if folder != 'backup':
                attack_results_path = f'{base}/{folder}/results.csv'
                eval_results_path = f'{base}/{folder}/eval_metric.json'
                df = pd.read_csv(attack_results_path)
                df.rename(columns={'generated_sequence':'adversarial_sequence'}, inplace=True)
                metrics = calculate_metrics(model, lm_model, reader, w,
                                        df['label'].values, 
                                        df['sequence'].values,
                                        df['adversarial_sequence'].values)
                json.dump(metrics, open(eval_results_path, 'w'))
                #break
        #break

In [100]:
metrics

{'accuracy_drop': 0.38999999999999996,
 'roc_auc_drop': 0.5020128824476651,
 'probability_drop': 0.17572516789356185,
 'WER': 5.49,
 'NAD': 0.12517686790392343,
 'NAD_new': 0.19143180001522453,
 'NAD_new/log_ppl_adv': 0.049579214656884814,
 'NAD_new*log_ppl_ratio': 0.19864671167377868,
 'log_ppl_adv': 4.227407988330665,
 'log_ppl_orig': 4.559984352417324}

In [149]:
for dataset in ['kaggle_transactions', 'ag_news', 'insurance', 'ai_academy_data']:
#for dataset in ['ag_news']:
#for dataset in ['ai_academy_data']:
#for dataset in ['insurance']:
#for dataset in ['kaggle_transactions']:
    base_dir = f'results/{dataset}'
    model_with_weights = joblib.load(f'experiments/{dataset}/lstm/model')
    model = model_with_weights['model']
    w = model_with_weights['weights']
    reader = LanguageModelingReader()
    language_model_path = f'experiments/lm_models/{dataset}'
    lm_path = os.path.join(language_model_path, 'best.th')
    lm_vocab_path = os.path.join(language_model_path, 'vocab')
    vocab = Vocabulary.from_files(lm_vocab_path)
    lm_model = get_model_by_name('lm', vocab=vocab)
    load_weights(lm_model, lm_path)

    #for method in ['mcmc', 'random', 'cascada', 'hotflip']:
    for method in ['hotflip']:
        base = os.path.join(base_dir, method)
        for folder in os.listdir(base):
            if folder != 'backup':
                attack_results_path = f'{base}/{folder}/results.csv'
                eval_results_path = f'{base}/{folder}/eval_metric_lstm.json'
                df = pd.read_csv(attack_results_path)
                df.rename(columns={'generated_sequence':'adversarial_sequence'}, inplace=True)
                metrics = calculate_metrics(model, lm_model, reader, w,
                                        df['label'].values, 
                                        df['sequence'].values,
                                        df['adversarial_sequence'].values)
                json.dump(metrics, open(eval_results_path, 'w'))
                #break
        #break

In [104]:
metrics

{'accuracy_drop': 0.15000000000000002,
 'roc_auc_drop': 0.17817496116499598,
 'probability_drop': 0.08802118686959147,
 'WER': 1.01,
 'NAD': 0.8636355000008635,
 'NAD_new': 0.86363624406681,
 'NAD_new/log_ppl_adv': 0.18536451228873024,
 'NAD_new*log_ppl_ratio': 0.7324584086731207,
 'log_ppl_adv': 4.715265071246018,
 'log_ppl_orig': 3.9384553467954873}

In [ ]:
# EVAL_FILENAME = 'eval_metric.json'

In [103]:
EVAL_FILENAME = 'eval_metric_lstm.json'

In [225]:
#dataset = 'kaggle_transactions'
#dataset = 'ag_news' 
#dataset = 'insurance'
dataset = 'ai_academy_data'


base_dir = f'results/{dataset}'


In [226]:
result = []
for method in ['random']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            try:
                args = json.load(open(os.path.join(base, folder, 'args.json')))
                metrics = json.load(open(os.path.join(base, folder, EVAL_FILENAME)))
                args.update(metrics)
                result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD', 
                                                   'NAD_new', 'NAD_new/log_ppl_adv', 'NAD_new*log_ppl_ratio', 'log_ppl_orig', 'log_ppl_adv', 
                                                    'beam_size', 'num_steps', 'std', 'space']})
            except Exception as e:
                print(e)
result = pd.DataFrame(result)

In [227]:
print('random')
result.sort_values(by='NAD_new', ascending=False)

random


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,beam_size,num_steps,std,space
11,0.318137,0.38,0.143336,4.18,0.267452,0.552920,0.147147,0.557578,3.938455,3.792000,1,100,0.100,decoder_hidden
1,0.245820,0.30,0.121764,4.86,0.188177,0.360721,0.104221,0.395830,3.938455,3.502621,30,30,0.010,encoder_outputs
3,0.289633,0.28,0.135285,4.84,0.170707,0.337651,0.098943,0.369801,3.938455,3.461348,30,30,0.100,decoder_hidden
7,0.313682,0.29,0.154503,6.05,0.144635,0.320176,0.091709,0.349389,3.938455,3.590110,30,30,0.100,encoder_outputs
8,0.328035,0.37,0.162664,6.43,0.150243,0.313156,0.095633,0.370931,3.938455,3.519587,30,30,1.000,decoder_hidden
9,0.255264,0.26,0.123378,4.94,0.147457,0.269865,0.077157,0.292683,3.938455,3.500648,30,30,0.010,decoder_hidden
0,0.388702,0.37,0.181830,8.87,0.108465,0.225950,0.054675,0.210903,3.938455,4.293353,1,100,1.000,decoder_hidden
2,0.386603,0.29,0.184638,8.24,0.109196,0.225717,0.060929,0.244033,3.938455,3.937227,30,30,1.000,encoder_outputs
6,0.220144,0.26,0.108407,5.67,0.130006,0.222056,0.063759,0.246091,3.938455,3.502912,30,30,0.001,encoder_outputs
5,0.099058,0.15,0.056823,1.32,0.107792,0.217628,0.053851,0.236770,3.938455,3.770820,1,100,0.010,decoder_hidden


In [228]:
print('random')
result[result['WER'] < 2].sort_values(by='NAD_new', ascending=False)

random


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,beam_size,num_steps,std,space
5,0.099058,0.15,0.056823,1.32,0.107792,0.217628,0.053851,0.236770,3.938455,3.770820,1,100,0.010,decoder_hidden
4,0.035151,0.10,0.016537,1.35,0.065557,0.128126,0.030590,0.130595,3.938455,3.764719,1,100,0.001,decoder_hidden


In [229]:
dataset

'ai_academy_data'

In [230]:
#EVAL_FILENAME = 'eval_metric.json'
EVAL_FILENAME = 'eval_metric_lstm.json'

In [148]:
pd.concat(all_res)

,dataset,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv
0,ag_news,0.341560,0.67,0.477214,1.40,0.611111,0.723367,0.108940,0.703169,6.562890,6.763453
0,insurance,0.777778,0.92,0.293957,2.53,0.448148,0.371613,0.058434,0.193973,3.128740,6.753455
0,ai_academy_data,0.019505,0.00,0.025177,1.05,0.319149,0.319149,0.067516,0.274808,3.938455,4.830067
0,kaggle_transactions,0.243156,0.26,0.090688,1.01,0.623188,0.623188,0.129443,0.547102,4.559984,5.148704


In [152]:
# lstm
pd.concat(all_res)

,dataset,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv
0,ag_news,0.336205,0.85,0.621411,1.40,0.754578,0.873020,0.133203,0.849258,6.562890,6.763453
0,insurance,0.494949,0.96,0.713257,2.53,0.475429,0.414637,0.065587,0.221903,3.128740,6.753455
0,ai_academy_data,0.263662,0.14,0.124760,1.05,0.897726,0.926110,0.193256,0.755620,3.938455,4.830067
0,kaggle_transactions,0.406200,0.35,0.246203,1.01,0.977272,0.983901,0.195586,0.873645,4.559984,5.148704


In [136]:
pd.concat(all_res)

,dataset,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv
0,kaggle_transactions,0.330113,0.31,0.199514,1.0,0.939393,0.939394,0.186624,0.829695,4.559984,5.135321


In [231]:
result = []
for method in ['mcmc']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            try:
                args = json.load(open(os.path.join(base, folder, 'args.json')))
                metrics = json.load(open(os.path.join(base, folder, EVAL_FILENAME)))
                args.update(metrics)
                result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                                    'NAD_new', 'NAD_new/log_ppl_adv', 
                                                   'NAD_new*log_ppl_ratio', 'log_ppl_orig', 'log_ppl_adv',
                                                   'beam_size', 'num_steps', 'std', 'sigma_wer', 'sigma_class']})
            except Exception as e:
                print(e)
result = pd.DataFrame(result)

In [232]:
print('mcmc')
result[(result['WER'] < 2)&(result['beam_size'] == 30)&(result['num_steps'] == 30)].sort_values(by='NAD_new', ascending=False).head()

mcmc


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,beam_size,num_steps,std,sigma_wer,sigma_class
54,0.113710,0.15,0.063995,1.60,0.065881,0.121558,0.034479,0.128652,3.938455,3.849192,30,30,0.010,1.5,0.20
23,0.084789,0.10,0.037517,1.42,0.059307,0.109930,0.033124,0.112567,3.938455,3.842050,30,30,0.100,0.2,1.00
25,0.083719,0.11,0.042937,1.99,0.064664,0.107346,0.031452,0.115746,3.938455,3.771509,30,30,0.001,0.2,1.50
20,0.093077,0.08,0.043746,1.69,0.056818,0.087519,0.025951,0.085391,3.938455,3.855643,30,30,0.100,0.2,1.25
45,0.092650,0.14,0.046236,1.53,0.043118,0.068153,0.018541,0.069472,3.938455,3.980264,30,30,1.000,0.2,1.25


In [233]:
print('mcmc')
result[(result['WER'] < 2)].sort_values(by='NAD_new', ascending=False).head()

mcmc


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,beam_size,num_steps,std,sigma_wer,sigma_class
54,0.113710,0.15,0.063995,1.60,0.065881,0.121558,0.034479,0.128652,3.938455,3.849192,30,30,0.010,1.5,0.20
23,0.084789,0.10,0.037517,1.42,0.059307,0.109930,0.033124,0.112567,3.938455,3.842050,30,30,0.100,0.2,1.00
25,0.083719,0.11,0.042937,1.99,0.064664,0.107346,0.031452,0.115746,3.938455,3.771509,30,30,0.001,0.2,1.50
20,0.093077,0.08,0.043746,1.69,0.056818,0.087519,0.025951,0.085391,3.938455,3.855643,30,30,0.100,0.2,1.25
45,0.092650,0.14,0.046236,1.53,0.043118,0.068153,0.018541,0.069472,3.938455,3.980264,30,30,1.000,0.2,1.25


In [234]:
print('mcmc')
result.sort_values(by='NAD_new/log_ppl_adv', ascending=False).head()

mcmc


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,beam_size,num_steps,std,sigma_wer,sigma_class
19,0.312077,0.34,0.141602,5.33,0.177849,0.345567,0.102295,0.382958,3.938455,3.524235,30,30,0.10,1.00,1.00
34,0.325259,0.34,0.151437,5.42,0.165396,0.350339,0.100421,0.386169,3.938455,3.526520,30,30,0.10,2.00,0.75
16,0.271955,0.32,0.130628,5.00,0.192984,0.335357,0.096993,0.361796,3.938455,3.508525,30,30,0.01,2.00,1.50
6,0.296322,0.33,0.138730,5.51,0.162807,0.334684,0.096895,0.362479,3.938455,3.552113,30,30,0.10,1.25,0.75
2,0.319100,0.34,0.143960,5.84,0.165287,0.330567,0.094785,0.354816,3.938455,3.559739,30,30,0.10,1.00,1.25


In [235]:
result = []
for method in ['cascada']:
    base = os.path.join(base_dir, method)
    for folder in os.listdir(base):
        if folder != 'backup':
            args = json.load(open(os.path.join(base, folder, 'args.json')))
            metrics = json.load(open(os.path.join(base, folder, EVAL_FILENAME)))
            args.update(metrics)
            result.append({k:args[k] for k in ['roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD',
                                               'NAD_new', 'NAD_new/log_ppl_adv', 'NAD_new*log_ppl_ratio', 'log_ppl_orig', 
                                               'log_ppl_adv',
                                               'levenshtein_weight', 'beam_size', 'max_steps', 'learning_rate']})
result = pd.DataFrame(result)

In [236]:
print('cascada')
result[(result['WER'] < 2)&(result['beam_size'] == 30)&(result['max_steps'] == 30)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,levenshtein_weight,beam_size,max_steps,learning_rate
8,0.308129,0.35,0.139339,1.63,0.529852,0.662272,0.163651,0.626571,3.938455,4.050231,0.1,30,30,1.0
20,0.286991,0.31,0.136027,1.41,0.420075,0.574424,0.153330,0.569281,3.938455,3.953288,1.0,30,30,1.0
35,0.277648,0.30,0.134912,1.63,0.396573,0.517924,0.137252,0.514032,3.938455,3.984694,10.0,30,30,1.0


In [237]:
print('cascada')
result[(result['WER'] < 9)&(result['beam_size'] == 30)&(result['max_steps'] == 30)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,levenshtein_weight,beam_size,max_steps,learning_rate
8,0.308129,0.35,0.139339,1.63,0.529852,0.662272,0.163651,0.626571,3.938455,4.050231,0.1,30,30,1.0
20,0.286991,0.31,0.136027,1.41,0.420075,0.574424,0.153330,0.569281,3.938455,3.953288,1.0,30,30,1.0
35,0.277648,0.30,0.134912,1.63,0.396573,0.517924,0.137252,0.514032,3.938455,3.984694,10.0,30,30,1.0
14,0.437556,0.36,0.207100,4.13,0.356079,0.495181,0.126970,0.471733,3.938455,4.158728,1.0,30,30,2.0
5,0.437887,0.37,0.206984,4.73,0.359790,0.469301,0.119556,0.444460,3.938455,4.321934,0.1,30,30,2.0
21,0.443197,0.36,0.210446,4.47,0.345134,0.451435,0.114291,0.423055,3.938455,4.238608,10.0,30,30,2.0


In [238]:
print('cascada')
result[(result['WER'] < 2)].sort_values(by='NAD_new', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,levenshtein_weight,beam_size,max_steps,learning_rate
16,0.288808,0.35,0.132331,1.40,0.563095,0.689516,0.171403,0.656077,3.938455,4.015508,0.1,30,100,1.0
8,0.308129,0.35,0.139339,1.63,0.529852,0.662272,0.163651,0.626571,3.938455,4.050231,0.1,30,30,1.0
30,0.283060,0.31,0.134603,1.22,0.430898,0.583162,0.156587,0.578179,3.938455,3.939012,1.0,30,100,1.0
20,0.286991,0.31,0.136027,1.41,0.420075,0.574424,0.153330,0.569281,3.938455,3.953288,1.0,30,30,1.0
4,0.284478,0.30,0.135494,1.48,0.397421,0.521351,0.137836,0.516789,3.938455,3.973291,10.0,30,100,1.0
35,0.277648,0.30,0.134912,1.63,0.396573,0.517924,0.137252,0.514032,3.938455,3.984694,10.0,30,30,1.0
13,0.273823,0.27,0.130152,1.93,0.205961,0.396703,0.102756,0.393296,3.938455,3.883464,1.0,1,100,1.0
1,0.261399,0.24,0.122862,1.92,0.213343,0.385598,0.100952,0.397058,3.938455,3.917309,0.1,1,100,1.0
27,0.225124,0.16,0.112356,1.93,0.083368,0.186547,0.046607,0.186232,3.938455,3.941797,10.0,1,100,1.0


In [239]:
print('cascada')
result[(result['WER'] < 9)].sort_values(by='NAD_new/log_ppl_adv', ascending=False)

cascada


,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv,levenshtein_weight,beam_size,max_steps,learning_rate
16,0.288808,0.35,0.132331,1.40,0.563095,0.689516,0.171403,0.656077,3.938455,4.015508,0.1,30,100,1.0
8,0.308129,0.35,0.139339,1.63,0.529852,0.662272,0.163651,0.626571,3.938455,4.050231,0.1,30,30,1.0
30,0.283060,0.31,0.134603,1.22,0.430898,0.583162,0.156587,0.578179,3.938455,3.939012,1.0,30,100,1.0
20,0.286991,0.31,0.136027,1.41,0.420075,0.574424,0.153330,0.569281,3.938455,3.953288,1.0,30,30,1.0
4,0.284478,0.30,0.135494,1.48,0.397421,0.521351,0.137836,0.516789,3.938455,3.973291,10.0,30,100,1.0
35,0.277648,0.30,0.134912,1.63,0.396573,0.517924,0.137252,0.514032,3.938455,3.984694,10.0,30,30,1.0
7,0.427320,0.36,0.204812,4.03,0.356876,0.501295,0.129157,0.481354,3.938455,4.150009,1.0,30,100,2.0
14,0.437556,0.36,0.207100,4.13,0.356079,0.495181,0.126970,0.471733,3.938455,4.158728,1.0,30,30,2.0
15,0.441625,0.37,0.206695,4.32,0.371267,0.491634,0.124763,0.462745,3.938455,4.311465,0.1,30,100,2.0
5,0.437887,0.37,0.206984,4.73,0.359790,0.469301,0.119556,0.444460,3.938455,4.321934,0.1,30,30,2.0


In [240]:
dataset

'ai_academy_data'

In [222]:
dataset

'ag_news'

In [242]:
df

,sequence,label,adversarial_sequence,adversarial_label,wer,prob_diff,acceptance_probability
0,belgians find canadian jokes about them less ...,0,interstellar find bryant with them less than c...,3,6,-0.020722,NaN
1,vodafone targets ipod,3,henman tops raptors,1,3,0.857205,NaN
2,leaders reactions mixed on possible legoland...,2,leaders for mixed on possible ap,0,3,0.164708,NaN
3,ibm recalls laptop power units worldwide,3,ibm recalls laptop power dividend worldwide,2,1,0.423536,NaN
4,congress finishing up b spending bill ap,0,congress to up spyware standard ap,3,4,0.941553,NaN
...,...,...,...,...,...,...,...
95,indias most wanted bandit killed in shootout ...,0,impressive most wanted bandit killed in shootout,0,2,0.182907,NaN
96,sony shrinking the ps,3,marsh shrinking the ukraine,2,2,0.965626,NaN
97,wall street treads water as traders await fre...,2,wall street says water traders await fresh ukr...,0,3,0.680491,NaN
98,federal judge orders soldier back to iraq,0,federal judge orders soldier back to jobs,2,1,0.552947,NaN


In [61]:
all_metrics = []
all_seqs = []
for dataset in ['ag_news']:
    base_dir = f'results/{dataset}'
    model_with_weights = joblib.load(f'results/{dataset}/logit_tfidf.model')
    model = model_with_weights['model']
    w = model_with_weights['weights']
    reader = LanguageModelingReader()
    language_model_path = f'experiments/lm_models/{dataset}'
    lm_path = os.path.join(language_model_path, 'best.th')
    lm_vocab_path = os.path.join(language_model_path, 'vocab')
    vocab = Vocabulary.from_files(lm_vocab_path)
    lm_model = get_model_by_name('lm', vocab=vocab)
    load_weights(lm_model, lm_path)
    for method in ['cascada_masked']:
        base = os.path.join(base_dir, method)
        for folder in sorted(os.listdir(base)):
            if folder != 'backup':
                attack_results_path = f'{base}/{folder}/results.csv'
                eval_results_path = f'{base}/{folder}/eval_metric.json'
                args_path = f'{base}/{folder}/args.json'
                df = pd.read_csv(attack_results_path)
                df.rename(columns={'generated_sequence':'adversarial_sequence'}, inplace=True)
                metrics = calculate_metrics(model, lm_model, reader, w,
                                        df['label'].values, 
                                        df['sequence'].values,
                                        df['adversarial_sequence'].values)
                json.dump(metrics, open(eval_results_path, 'w'))
                args = json.load(open(args_path, 'r'))
                print(attack_results_path)
                print(args['maskers'])
                print(metrics)
                print('')
                metrics.update(args)
                all_metrics.append(metrics)
                adv_seq = df[['adversarial_sequence']].copy()
                adv_seq.columns = [f'adversarial_sequence_{args["maskers"]}']
                all_seqs.append(adv_seq)
orig_seq = df[['sequence']].copy()
all_seqs.append(orig_seq)

results/ag_news/cascada_masked/20200220_183427/results.csv
Identity
{'accuracy_drop': 0.62, 'roc_auc_drop': 0.4219386242612946, 'probability_drop': 0.4922277125741614, 'WER': 2.51, 'NAD': 0.34254220345025027, 'NAD_new': 0.49231881337950595, 'NAD_new/log_ppl_adv': 0.07912610956110433, 'NAD_new*log_ppl_ratio': 0.5108304080652103, 'log_ppl_adv': 6.325082243690557, 'log_ppl_orig': 6.562889836294282}

results/ag_news/cascada_masked/20200220_183852/results.csv
Identity
{'accuracy_drop': 0.5700000000000001, 'roc_auc_drop': 0.39884539334107694, 'probability_drop': 0.4475081903384297, 'WER': 3.06, 'NAD': 0.2718526077765105, 'NAD_new': 0.38642574215768, 'NAD_new/log_ppl_adv': 0.05986014870468968, 'NAD_new*log_ppl_ratio': 0.38669905795685094, 'log_ppl_adv': 6.584105536912003, 'log_ppl_orig': 6.562889836294282}

results/ag_news/cascada_masked/20200220_184808/results.csv
RemoveMasker
{'accuracy_drop': 0.62, 'roc_auc_drop': 0.4191204757135667, 'probability_drop': 0.4720054983556253, 'WER': 4.08, 'NA

In [63]:
all_metrics = pd.DataFrame(all_metrics)

In [65]:
all_metrics['maskers'].values[0] = 'Non-masked'

In [66]:
all_metrics[['maskers', 'roc_auc_drop', 'accuracy_drop', 'probability_drop', 'WER', 'NAD', 'NAD_new', 'NAD_new/log_ppl_adv', 'log_ppl_adv', 'NAD_new*log_ppl_ratio', 'log_ppl_orig', 'log_ppl_adv']]

,maskers,roc_auc_drop,accuracy_drop,probability_drop,WER,NAD,NAD_new,NAD_new/log_ppl_adv,log_ppl_adv,NAD_new*log_ppl_ratio,log_ppl_orig,log_ppl_adv
0,Non-masked,0.421939,0.62,0.492228,2.51,0.342542,0.492319,0.079126,6.325082,0.510830,6.56289,6.325082
1,Identity,0.398845,0.57,0.447508,3.06,0.271853,0.386426,0.059860,6.584106,0.386699,6.56289,6.584106
2,RemoveMasker,0.419120,0.62,0.472005,4.08,0.191678,0.317368,0.052450,6.390435,0.331165,6.56289,6.390435
3,ReplaceMasker,0.418795,0.59,0.466093,3.21,0.248951,0.366393,0.060599,6.299338,0.391416,6.56289,6.299338
4,TwoSwapMasker,0.466164,0.61,0.494952,3.36,0.237498,0.332800,0.054946,6.258652,0.351877,6.56289,6.258652
5,AddMaskerrandom,0.435654,0.63,0.486403,3.28,0.250000,0.381691,0.061171,6.366830,0.387882,6.56289,6.366830


In [265]:
all_seqs = pd.concat(all_seqs, axis=1)

In [266]:
all_seqs.columns.values[0] = 'adversarial_sequence_nonmasked'

In [267]:
all_seqs = all_seqs[['sequence'] + all_seqs.columns.values[:-1].tolist()]

In [268]:
all_seqs.to_excel(f'results/{dataset}/ag_news_masked_examples.xlsx', index=False)